In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os, time, string
import pandas as pd
import warnings
import openpyxl
import numpy as np
from openpyxl.utils import get_column_letter
warnings.filterwarnings('ignore')
%autosave 120

Autosaving every 120 seconds


### 產出玩具資料

In [2]:
# 玩具資料1 欄位：身份證號、性別、就醫年份、年齡層
n = 1000
year1 = ['2020','2021','2022']
year_t = list(np.random.choice(list(year1),n,replace = True))

age1 = ['0-4','5-11','12-17','18-65']
age_t = list(np.random.choice(list(age1),n,replace = True))

id1 = list(np.random.randint(100000000,299999999,n))
id2 = list(np.random.choice(list(string.ascii_uppercase),n,replace = True))
id3 = [id2[i] + str(id1[i]) for i in range(n)]

sex1 = ['男','女']
sex_t = ['男' if i[1] == '1' else '女' for i  in id3]

dict1 = {'身份證號':id3,'性別':sex_t,'就醫年份':year_t,'年齡層':age_t}
df = pd.DataFrame(dict1)
df1 = df.copy()

# 玩具資料2 欄位：身份證號、縣市、區域
county = ['01基隆市','02臺北市','03新北市','04宜蘭縣','05金門縣','06連江縣','07桃園市','08新竹市','09新竹縣','10苗栗縣','11台中市','12南投縣','13彰化縣','14雲林縣','15嘉義市','16嘉義縣','17台南市','18高雄市','19屏東縣','20澎湖縣','21花蓮縣','22台東縣'] 
col1_data = np.random.choice(county, 1000, replace=True) #縣市
col2_data = id3 #身份證
dict2 = {'身份證號':id3,'縣市':col1_data}
df = pd.DataFrame(dict2)
df2 = df.copy()
def define_region(x):
    if x in ['01基隆市','02臺北市','03新北市','04宜蘭縣','05金門縣','06連江縣']:
        return('01台北區')
    elif x in ['07桃園市','08新竹市','09新竹縣','10苗栗縣']:
        return('02北區')
    elif x in ['11台中市','12南投縣','13彰化縣']:
        return('03中區')
    elif x in ['14雲林縣','15嘉義市','16嘉義縣','17台南市']:
        return('04南區')
    elif x in ['18高雄市','19屏東縣','20澎湖縣']:
        return('05高屏區')
    elif x in ['21花蓮縣','22台東縣']:
        return('06東區')
    
df2['區域'] = df2['縣市'].apply(define_region)

In [5]:
df2

,身份證號,縣市,區域
0,J164570883,09新竹縣,02北區
1,M271744653,18高雄市,05高屏區
2,Z174718127,04宜蘭縣,01台北區
3,J227102825,16嘉義縣,04南區
4,I130533355,05金門縣,01台北區
...,...,...,...
995,R209940653,01基隆市,01台北區
996,E293639891,15嘉義市,04南區
997,O255357111,22台東縣,06東區
998,Z228848123,07桃園市,02北區


In [3]:
# 將df2 轉為樞紐表 df2_pivot_table 並除去縣市及區域的數字 例如： 01台北區 -> 台北區
df2_pivot_table = df2.pivot_table(index=['區域', '縣市'], values='身份證號',aggfunc='count')
df2_ind = df2_pivot_table.index
df2_ind_new = [(i[0][2:],i[1][2:]) for i in df2_ind]
df2_index = pd.MultiIndex.from_tuples(df2_ind_new, names=["區域", "縣市"])
df2_pivot_table.index = df2_index
df2_pivot_table['身份證號'] = df2_pivot_table['身份證號'].apply(lambda x: x/1000)
df2_pivot_table['平均']=round(df2_pivot_table['身份證號'].mean(),3)
df_tmp = pd.DataFrame({
    "身份證號": [1],
    "平均": [1]
}, index=[('總計','')])
# Add specific row/index name using DataFrame.loc[]
df2_pivot_table = df2_pivot_table.append(df_tmp)
# display(df2_pivot_table)

In [4]:
display(df2_pivot_table)

身份證號     平均
區域  縣市               
台北區 基隆市  0.061  0.045
    臺北市  0.045  0.045
    新北市  0.042  0.045
    宜蘭縣  0.049  0.045
    金門縣  0.051  0.045
    連江縣  0.049  0.045
北區  桃園市  0.031  0.045
    新竹市  0.059  0.045
    新竹縣  0.043  0.045
    苗栗縣  0.048  0.045
中區  台中市  0.046  0.045
    南投縣  0.033  0.045
    彰化縣  0.050  0.045
南區  雲林縣  0.050  0.045
    嘉義市  0.039  0.045
    嘉義縣  0.055  0.045
    台南市  0.033  0.045
高屏區 高雄市  0.044  0.045
    屏東縣  0.039  0.045
    澎湖縣  0.045  0.045
東區  花蓮縣  0.039  0.045
    台東縣  0.049  0.045
總計       1.000  1.000

In [7]:
# 儲存df2_pivot_table 至 eccel檔
address_test = r'D:\Users\mik986407\py_data\hiv_output\測試2.xlsx'
df2_pivot_table.to_excel(address_test)

### 畫圖1： 以 openpyxl 繪圖（舊版）
 參考網站： https://openpyxl.readthedocs.io/en/stable/charts/introduction.html#

In [9]:
from openpyxl.chart import (
    Reference,
    Series,
    BarChart,
    LineChart
)

#匯入資料
address_test = r'D:\Users\mik986407\py_data\hiv_output\測試2.xlsx'
wb = openpyxl.load_workbook(address_test)# 開啟 Excel 檔案
sheetnames = wb.sheetnames
s1 = wb[sheetnames[0]]

#定義資料範圍
data = Reference(s1, min_col=3, min_row=1, max_row=23)
titles = Reference(s1, min_col=1, min_row=2,max_col=2 ,max_row=23)
chart = BarChart()

#畫圖
chart.title = "Bar Chart"
chart.type = "col"
chart.style = 4
chart.add_data(data=data, titles_from_data=True)
chart.set_categories(titles)
chart.legend.position = 'b'
c1 = chart
# Create a second chart
c2 = LineChart()
v2 = Reference(s1, min_col=4, min_row=2, max_row=23)
c2.add_data(v2, titles_from_data=False)
s2 = c2.series[0]
s2.graphicalProperties.line.solidFill = "FF0000"
s2.graphicalProperties.line.dashStyle = "sysDot"
c1 += c2

s1.add_chart(c1, "G1")
wb.save(address_test)

### 畫圖2 :以 xlsxwriter 繪圖 （新版）
 參考網站： https://xlsxwriter.readthedocs.io/index.html

In [30]:
import xlsxwriter

In [35]:
def plot_chart(workbook,sheet_name,color):
    # Create a new bar chart.
    chart = workbook.add_chart({'type': 'column'})
    # Configure the first series.
    chart.add_series({
        'name':       '={}!$C$1'.format(sheet_name),
        'categories': '={}!$A$2:$B$23'.format(sheet_name),
        'values':     '={}!$C$2:$C$23'.format(sheet_name),
        'fill':   {'color': '{}'.format(color)}
    })
    # Add a chart title and some axis labels in sheet1.
    chart.set_title ({'name': 'Results of sample analysis'})
    chart.set_x_axis({'name': ''})
    chart.set_y_axis({'name': ''})
    # Create a secondary chart.
    line_chart = workbook.add_chart({'type': 'line'})
    custom_labels = [
        {'delete': True},{'delete': True},{'delete': True},{'delete': True},{'delete': True},{'delete': True},
        {'delete': True},{'delete': True},{'delete': True},{'delete': True},{'delete': True},{'delete': True},
        {'delete': True},{'delete': True},{'delete': True},{'delete': True},{'delete': True},{'delete': True},
        {'delete': True},{'delete': True},{'delete': True},None
    ]
    line_chart.add_series({
    #     'name':       '=Sheet1!$D$1',
    #     'categories': '=Sheet1!$A$2:$B$23',
        'values':     '={}!$D$2:$D$23'.format(sheet_name),
        'data_labels': {'value': True, 'custom': custom_labels},
        'line':   {'dash_type': 'dash','width': 1.25,'color': 'red'}

    })
    # Combine the charts.
    chart.combine(line_chart)
    # Set an Excel chart style.
    chart.set_x_axis({'major_tick_mark': 'none'})
    chart.set_y_axis({'major_tick_mark': 'none','line': {'none': True},'num_format':'0%'})
    chart.set_style(5)
    chart.set_legend({'position': 'bottom','delete_series': [1]})
    return(chart)

In [37]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'D:\Users\mik986407\py_data\hiv_output\pivot_table_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
sheet_name = 'Sheet1'
sheet_name2 = 'Sheet2'
df2_pivot_table.to_excel(writer, sheet_name=sheet_name)
df2_pivot_table.to_excel(writer, sheet_name=sheet_name2)

# Access the XlsxWriter workbook and worksheet objects from the dataframe.
workbook  = writer.book
worksheet = writer.sheets[sheet_name]
# Write a conditional format over a range.
# Add a format1.
format1 = workbook.add_format({'font_color': '#9C0006'})
worksheet.conditional_format('C2:C23', {'type': 'cell',
                                         'criteria': '>=',
                                         'value': 0.045,
                                         'format': format1})
# Add a percent number format.
percent_format = workbook.add_format({'num_format': '0.0%'})

# Apply the number format to C,D columns.
worksheet.set_column('C:D', 23, percent_format)

sheet1_chart1 = plot_chart(workbook,sheet_name,'#FF9900')
sheet1_chart2 = plot_chart(workbook,sheet_name,'#007979')
sheet1_chart3 = plot_chart(workbook,sheet_name,'#00BB00')
sheet1_chart4 = plot_chart(workbook,sheet_name,'#E1E100')
# Insert the chart into the worksheet (with an offset).
worksheet.insert_chart('F2', sheet1_chart1, {'x_scale': 1, 'y_scale': 0.75})
worksheet.insert_chart('U2', sheet1_chart2, {'x_scale': 1, 'y_scale': 0.75})
worksheet.insert_chart('F24', sheet1_chart3, {'x_scale': 1, 'y_scale': 0.75})
worksheet.insert_chart('U24', sheet1_chart4, {'x_scale': 1, 'y_scale': 0.75})
#--------------------------------------------------------------------------------
# Access the XlsxWriter workbook and worksheet2 objects from the dataframe.
worksheet2 = writer.sheets[sheet_name2]
# Write a conditional format over a range.
# Add a format1.
format1 = workbook.add_format({'font_color': '#9C0006'})
worksheet2.conditional_format('C2:C23', {'type': 'cell',
                                         'criteria': '>=',
                                         'value': 0.045,
                                         'format': format1})

# Add a percent number format.
percent_format = workbook.add_format({'num_format': '0.0%'})

# Apply the number format to C,D columns.
worksheet2.set_column('C:D', 23, percent_format)

sheet2_chart1 = plot_chart(workbook,sheet_name,'#AD5A5A')
sheet2_chart2 = plot_chart(workbook,sheet_name,'#BB5E00')
sheet2_chart3 = plot_chart(workbook,sheet_name,'#9F4D95')
sheet2_chart4 = plot_chart(workbook,sheet_name,'#A8FF24')
# Insert the chart into the worksheet (with an offset).
worksheet2.insert_chart('F2', sheet2_chart1, {'x_scale': 1, 'y_scale': 0.75})
worksheet2.insert_chart('U2', sheet2_chart2, {'x_scale': 1, 'y_scale': 0.75})
worksheet2.insert_chart('F24', sheet2_chart3, {'x_scale': 1, 'y_scale': 0.75})
worksheet2.insert_chart('U24', sheet2_chart4, {'x_scale': 1, 'y_scale': 0.75})
writer.close()

In [ ]:
writer.close()

### 畫圖3 以seaborn考慮(測試中)

In [6]:
import seaborn as sns

In [7]:
df_test = sns.load_dataset("titanic")

In [20]:
df_test1 = df_test[['age','class']]
df_test1
sns.catplot(data=df_test1, x="age", y="class", kind="box")

In [29]:
df_test2 = df_test[['who','survived','age','class']]

In [26]:
df2
df2['累計'] = 1

In [22]:
sns.catplot(
    data=df_test, x="who", y="survived", col="class",
    kind="bar", height=4, aspect=.6
)
g.set_axis_labels("", "Survival Rate")
g.set_xticklabels(["Men", "Women", "Children"])
g.set_titles("{col_name} {col_var}")
g.set(ylim=(0, 1))
g.despine(left=True)
g.figure.savefig(r"D:\Users\mik986407\py_data\hiv_output\hiv_plot\out.png")
# fig.savefig(r"D:\Users\mik986407\py_data\hiv_output\hiv_plot\out.png") 

In [28]:
g1=sns.catplot(
    data=df2, x="縣市", y="累計", col="區域",
    kind="bar", height=4, aspect=.6
)
# g.set_axis_labels("", "Survival Rate")
# g.set_xticklabels(["Men", "Women", "Children"])
# g.set_titles("{col_name} {col_var}")
# g.set(ylim=(0, 1))
# g.despine(left=True)
g1.figure.savefig(r"D:\Users\mik986407\py_data\hiv_output\hiv_plot\out2.png")

### 嘗試修改ppt裡table的text值

In [14]:
!pip install python-pptx

     ---------------------------------------- 10.1/10.1 MB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470935 sha256=809b92bbe2d0a13a8fb7d7a1f3899a75db8197b68ad438b8a74bdc253a8caaf4
  Stored in directory: d:\users\mik986407\appdata\local\pip\cache\wheels\0e\4a\ed\9653bc799915f52dce3f04d14946fbd85cce9c3cdedc9cfa71
Successfully built python-pptx


In [15]:
from pptx import Presentation
from pptx.enum.text import PP_ALIGN
from pptx.util import Pt
from pptx.dml.color import ColorFormat, RGBColor


prs = Presentation(r'D:\Users\mik986407\py_data\hiv_output\111-12組務會議.pptx')
slide = prs.slides[1] #first slide
for i in slide.shapes:
    if i.has_table:
        i.table.cell(1,2).text  = '4'
        a  = i.table.cell(1,2).text_frame.paragraphs[0]
        a.font.size = Pt(12)
        a.font.color.rgb = RGBColor(255, 0, 0)
        a.alignment = PP_ALIGN.RIGHT
            

prs.save(r'D:\Users\mik986407\py_data\hiv_output\111-12組務會議.pptx')
print('修改完畢')

修改完畢


In [ ]:
prs = Presentation(r'D:\Users\mik986407\py_data\hiv_output\111-12組務會議.pptx')
slide = prs.slides[1] #first slide
for i in slide.shapes:
    if i.has_table:
        print(i.table.cell(7,0).text)

### 將excel圖表匯入ppt

In [47]:
# 進階版(測試)將同一分頁的excel圖物件匯入至ppt
import os
import win32com.client as client
from win32com.client import constants        

xl = client.gencache.EnsureDispatch('Excel.Application')
ppt = client.gencache.EnsureDispatch('PowerPoint.Application')

wb = xl.Workbooks.Open(os.path.abspath(r'D:\Users\mik986407\py_data\hiv_output\pivot_table_simple.xlsx'))
prs = ppt.Presentations.Open(os.path.abspath(r"D:\Users\mik986407\py_data\hiv_output\111-12組務會議.pptx"))

sheet = wb.Sheets('Sheet1')


Slide = prs.Slides.Add(prs.Slides.Count+1,constants.ppLayoutBlank) #選擇黏貼圖表至最後一頁
# 設定黏貼的位置
Left_inch = [100,500,100,500]
Top_inch = [100,100,250,250]
i = 0
for ch in sheet.ChartObjects():
    ch.Activate()
    ch.Copy()
    shp = Slide.Shapes.PasteSpecial(constants.ppPasteShape)
    shp.Left = Left_inch[i]
    shp.Top = Top_inch[i]
    i += 1
    
prs.Save()
prs.Close()   
wb.Close(False)

ppt.Quit()
xl.Quit()

In [43]:
ppt.Quit()
xl.Quit()

com_error: (-2147417848, '用戶端中斷了已啟動物件的連線。', None, None)

In [8]:
os.getcwd()

'D:\\Users\\mik986407\\python_code\\hiv專案'

In [11]:
os.chdir(r'D:\Users\mik986407\py_data\hiv_output')

D:\Users\mik986407\py_data\hiv_output


### 將excel 圖物件匯出為png圖檔

In [63]:
import win32com.client
import PIL
from PIL import ImageGrab, Image
import os
import sys

inputExcelFilePath = r'D:\Users\mik986407\py_data\hiv_output\pivot_table_simple.xlsx'
outputPNGImagePath = r'D:\Users\mik986407\py_data\hiv_output\\'

# This function extracts a graph from the input excel file and saves it into the specified PNG image path (overwrites the given PNG image)
def saveExcelGraphAsPNG(inputExcelFilePath, outputPNGImagePath):
    # Open the excel application using win32com
    o = win32com.client.Dispatch("Excel.Application")
    # Disable alerts and visibility to the user
    o.Visible = 0
    o.DisplayAlerts = 0
    # Open workbook
    wb = o.Workbooks.Open(inputExcelFilePath)

    # Extract first sheet
    sheet = o.Sheets(1)
    for n, shape in enumerate(sheet.Shapes):
        # Save shape to clipboard, then save what is in the clipboard to the file
        shape.Copy()
        image = ImageGrab.grabclipboard()
        length_x, width_y = image.size
        size = int(factor * length_x), int(factor * width_y)
        image_resize = image.resize(size, Image.ANTIALIAS)
        # Saves the image into the existing png file (overwriting) TODO ***** Have try except?
        outputPNGImage = outputPNGImagePath + str(n) + '.png'
        image_resize.save(outputPNGImage, 'png', quality=95, dpi=(300, 300))
        pass
    pass

    wb.Close(True)
    o.Quit()
factor = 1
saveExcelGraphAsPNG(inputExcelFilePath, outputPNGImagePath)

# 將Excel 的 table 黏貼至 ppt(不好用)

In [ ]:
import win32com.client

# 打开Excel文件
excel = win32com.client.Dispatch('Excel.Application')
workbook = excel.Workbooks.Open(r'D:\Users\mik986407\python_code\hiv專案\hiv_outcome\pivot_table_全部個案.xlsx')
worksheet = workbook.Worksheets[0]

# 选择表格并复制到剪贴板
table_range = worksheet.Range('A1:O24')
table_range.Copy()

# 打开PowerPoint文件
powerpoint = win32com.client.Dispatch('PowerPoint.Application')
presentation = powerpoint.Presentations.Open(r'D:\Users\mik986407\python_code\hiv專案\hiv_outcome\112-04組務會議.pptx')
slide = presentation.Slides.Add(1, 11)

# 将表格粘贴到PowerPoint幻灯片
table_shape = slide.Shapes.Paste()
table_shape.Left = 50
table_shape.Top = 100

# 保存并关闭文件
presentation.Save()
presentation.Close()
excel.Quit()